In [16]:
from object_detection.callable_implementation import write_csv
from object_detection.callable_implementation import process_image
from object_detection.callable_implementation import get_dt_info
from object_detection.callable_implementation import load_model
from object_detection.callable_implementation import load_tensor_img
from object_detection.render_video import images_to_video_ffmpeg

import requests
import cv2
import numpy as np
import imutils
import time
import os
import threading
from queue import Queue
import shutil
from tqdm.notebook import tqdm
import joblib
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [17]:
# load the model
model, vehicle_classes, device = load_model()
loaded_model = joblib.load("/home/brownjordan317/fall_2024/CSCI443/Github/CSCI-443/traffic_prediction_model.pkl")


In [18]:
input_folder = '/home/brownjordan317/fall_2024/CSCI443/Github/CSCI-443/object_detection/right'
output_folder = 'preds_30fps_right_labeled'
csv_name = 'preds_30fps_right_labeled.csv'

# create the output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [19]:
def add_counts(data, image, pred, output_folder, image_file):
    # Resize the image
    image = imutils.resize(image, width=800)

    # Get image dimensions
    height, width, _ = image.shape

    # Add white padding to the right side for annotations
    padding_width = 300  # Adjust as needed for the text
    padded_image = np.full((height, width + padding_width, 3), 255, dtype=np.uint8)  # White background
    padded_image[:, :width] = image  # Place the original image on the left

    # Create annotations
    annotations = [
        f"Time: {data['Time']}",
        f"Day: {data['Day of the week']}",
        f"Car Count: {data['CarCount']}",
        f"Bike Count: {data['BikeCount']}",
        f"Bus Count: {data['BusCount']}",
        f"Truck Count: {data['TruckCount']}",
        f"Total: {data['Total']}",
        f"Prediction: {pred}"
    ]

    # Set annotation settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    font_color = (0, 0, 0)  # Black text for contrast with white background
    line_type = cv2.LINE_AA

    # Determine the starting position for annotations (right side of the padded image)
    x_offset = width + 10  # Leave a small margin from the start of the padding
    y_offset = 30  # Initial vertical offset
    line_height = 25  # Space between each annotation

    # Add each annotation to the image
    for i, text in enumerate(annotations):
        y_position = y_offset + i * line_height
        cv2.putText(padded_image, text, (x_offset, y_position), font, font_scale, font_color, 1, line_type)

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Save the modified image
    output_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_path, padded_image)

    print(f"Image saved to {output_path}")

In [20]:
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

for image_file in tqdm(image_files):
    # Load the image
    image_path = os.path.join(input_folder, image_file)
    img, img_tensor = load_tensor_img(image_path, device)
    data, out_image = process_image(model, vehicle_classes, img, img_tensor, output_folder, output_filename=image_file)
    time, day = get_dt_info()
    data["Time"] = time
    data["Day of the week"] = day
    write_csv(data, csv_name)
    # send data to model
    headers = ['Time', 'Day of the week', 'CarCount', 'BikeCount', 'BusCount', 'TruckCount', 'Total']
    X_pred = [data.get(header, "") for header in headers]
    # convert to df
    X_pred = pd.DataFrame([X_pred], columns=headers)
    le = LabelEncoder()
    X_pred['Day of the week'] = le.fit_transform(X_pred['Day of the week'])

    pred = int(loaded_model.predict(X_pred)[0])
    
    if pred == 0:
        pred = "heavy"
    elif pred == 1:
        pred = "high"
    elif pred == 2:
        pred = "low"
    elif pred == 3:
        pred = "normal"

    add_counts(data, out_image, pred, output_folder, image_file)

  0%|          | 0/1438 [00:00<?, ?it/s]

Image saved to preds_30fps_right_labeled/frame_1297.png
Image saved to preds_30fps_right_labeled/frame_0893.png
Image saved to preds_30fps_right_labeled/frame_1175.png
Image saved to preds_30fps_right_labeled/frame_0717.png
Image saved to preds_30fps_right_labeled/frame_0363.png
Image saved to preds_30fps_right_labeled/frame_0470.png
Image saved to preds_30fps_right_labeled/frame_0682.png
Image saved to preds_30fps_right_labeled/frame_1326.png
Image saved to preds_30fps_right_labeled/frame_1087.png
Image saved to preds_30fps_right_labeled/frame_0778.png
Image saved to preds_30fps_right_labeled/frame_0063.png
Image saved to preds_30fps_right_labeled/frame_0996.png
Image saved to preds_30fps_right_labeled/frame_0385.png
Image saved to preds_30fps_right_labeled/frame_1117.png
Image saved to preds_30fps_right_labeled/frame_1032.png
Image saved to preds_30fps_right_labeled/frame_0344.png
Image saved to preds_30fps_right_labeled/frame_0274.png
Image saved to preds_30fps_right_labeled/frame_0